<a href="https://colab.research.google.com/github/neohack22/ebw3nt/blob/main/modelisation/tokenisation_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Les données brutes de l'IMDB (budgets, nombres de votes, critiques textuelles) sont souvent hétérogènes. Elles ne peuvent pas être injectées directement dans une IA sans une phase de structuration préalable.

Cette métamorphose de la donnée suit une séquence logique et rigoureuse au sein du notebook pour garantir la fiabilité du modèle final.

## Flux de Travail

L'objectif est de convertir un script brut en un "dataset" prêt pour l'apprentissage automatique.

1. Nettoyage : Suppression des bruits, comme les caractères spéciaux dans les résumés de films ou l'élimination des entrées dont les informations essentielles (comme le titre) sont absentes.
2. Normalisation : Mise en conformité des échelles.

Exemple : Un film peut avoir 2 000 000 de votes alors que sa note est de 8.5/10. Sans normalisation (ramener ces valeurs sur une échelle de 0 à 1), le modèle d'IA pourrait croire que le nombre de votes est 235 000 fois plus important que la note, ce qui fausserait totalement ses prédictions.

3. Structuration : Organisation finale des données en tableaux optimisés, permettant à l'IA de détecter efficacement les corrélations entre le genre d'un film et son succès critique.

<br>

* Qualité des prédictions : Une IA ne "comprend" pas le cinéma ; elle comprend les chiffres. Des données propres garantissent des résultats précis.
* Performance du modèle : Des données normalisées et bien structurées permettent un entraînement beaucoup plus rapide et efficace.

L'objectif est d'implémenter le modèle proposé par Yoon Kim, publié en 2014. L'article original peut être trouvé [ici](https://www.aclweb.org/anthology/D14-1181).
Bien sûr, il existe des implémentations PyTorch et TensorFlow sur le web. Elles sont plus ou moins correctes et efficaces.


Nous utilisons le même jeu de données qu'auparavant : chargement des données, construction du vocabulaire et préparation des données pour le modèle.


# Chargement des données

In [ ]:
import re
import numpy as np
import torch as th
import torch.autograd as ag
import torch.nn.functional as F
import torch.nn as nn
import random

th.manual_seed(1) # set the seed


def clean_str(string, tolower=True):
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " ", string) ## remove
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ", string) ## remove
    string = re.sub(r"\)", " ", string)## remove
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()


def loadTexts(filename, limit=-1):
    """
    Texts loader for imdb.
    If limit is set to -1, the whole dataset is loaded, otherwise limit is the number of lines
    """
    f = open(filename)
    dataset=[]
    line =  f.readline()
    cpt=1
    skip=0
    while line :
        cleanline = clean_str(line).split()
        if cleanline:
            dataset.append(cleanline)
        else:
            line = f.readline()
            skip+=1
            continue
        if limit > 0 and cpt >= limit:
            break
        line = f.readline()
        cpt+=1

    f.close()
    print("Load ", cpt, " lines from ", filename , " / ", skip ," lines discarded")
    return dataset



In [ ]:
import re
import numpy as np
import torch as th
import torch.autograd as ag
import torch.nn.functional as F
import torch.nn as nn
import random

th.manual_seed(1) # set the seed


def clean_str(string, tolower=True):
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " ", string) ## remove
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ", string) ## remove
    string = re.sub(r"\)", " ", string)## remove
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()


def loadTexts(filename, limit=-1):
    """
    Texts loader for imdb.
    If limit is set to -1, the whole dataset is loaded, otherwise limit is the number of lines
    """
    f = open(filename)
    dataset=[]
    line =  f.readline()
    cpt=1
    skip=0
    while line :
        cleanline = clean_str(line).split()
        if cleanline:
            dataset.append(cleanline)
        else:
            line = f.readline()
            skip+=1
            continue
        if limit > 0 and cpt >= limit:
            break
        line = f.readline()
        cpt+=1

    f.close()
    print("Load ", cpt, " lines from ", filename , " / ", skip ," lines discarded")
    return dataset



La plupart des algorithmes de traitement du langage naturel s'attendent à ce qu'un seul espace sépare les mots, et des espaces supplémentaires pourraient être interprétés à tort ou simplement causer des problèmes d'analyse.

Imaginez que le mot "Film" et le mot "film" soient traités comme deux mots différents par votre IA. Cela doublerait inutilement le vocabulaire et rendrait plus difficile pour l'IA d'apprendre que ces deux mots ont la même signification. En convertissant tout en minuscules, on s'assure que "Film", "FILM", et "film" sont tous traités comme un seul et même mot : "film". C'est une étape fondamentale pour réduire la complexité du vocabulaire et améliorer la performance des modèles de traitement du langage naturel.

Load the data

In [ ]:
LIM=-1
pathd = "/home/allauzen/cours/nlp-iasd/labs/"
txtfile=pathd+"imdb.pos"
postxt = loadTexts(txtfile,limit=LIM)
print(postxt[0:10])
print (len(postxt), " pos sentences")

txtfile=pathd+"imdb.neg"
negtxt = loadTexts(txtfile,limit=LIM)
print(negtxt[0:10])

print (len(negtxt), " neg sentences")


Load  299966  lines from  /home/allauzen/cours/nlp-iasd/labs/imdb.pos  /  35  lines discarded
[['excellent'], ['do', "n't", 'miss', 'it', 'if', 'you', 'can'], ['a', 'great', 'parody'], ['dreams', 'of', 'a', 'young', 'girl'], ['tromendous', 'piece', 'of', 'art'], ['funny', 'funny', 'movie', '!'], ['need', 'more', 'scifi', 'like', 'this'], ['pride', 'and', 'prejudice', 'is', 'absolutely', 'amazing', '!', '!'], ['scott', 'pilgrim', 'vs', 'the', 'world'], ['quirky', 'and', 'effective']]
299965  pos sentences
Load  299949  lines from  /home/allauzen/cours/nlp-iasd/labs/imdb.neg  /  52  lines discarded
[['typical', 'movie', 'where', 'best', 'parts', 'are', 'in', 'the', 'preview'], ['not', 'for', 'the', 'squeamish'], ['cool', 'when', 'i', 'was', 'kid'], ['i', 'appreciate', 'the', 'effort', 'but'], ['pretty', 'bad'], ['much', 'ado', 'about', 'nothing'], ['series', 'of', 'unlikely', 'events'], ['april', 'is', 'the', 'cruelest', 'month'], ['great', 'idea', 'but'], ['and', 'people', 'thought', 't

In [ ]:
wfreq = {}
maxlength = 0
for sent in postxt+negtxt:
    isent = []
    maxlength = max(maxlength,len(sent))
    for w in sent:
        if w in wfreq:
            wfreq[w] = wfreq[w]+1
        else :
            wfreq[w]=1 # On ajoute le mot à notre dictionnaire wfreq et on lui donne un compte de 1.

Les modèles d'IA ne peuvent pas traiter des phrases de longueurs arbitraires (une phrase de 5 mots, puis une de 50 mots, puis une de 12 mots) de manière directe. Ils ont besoin que toutes les séquences d'entrée aient la même longueur.

In [ ]:
print(len(wfreq)) # nombre total de mots uniques
orderedvocab = []
for w in sorted(wfreq, key=wfreq.get, reverse=True):
    orderedvocab.append((w, wfreq[w]))

63699


In [ ]:
print(orderedvocab[0:10])

[('!', 153714), ('the', 146409), ('a', 131821), ('of', 94543), ('movie', 80115), ('and', 63910), ('this', 53299), ('to', 46991), ('it', 46431), ('i', 44902)]


Ce code transforme notre liste de mots triés en un système de traduction bidirectionnel (mot <-> index) pour que l'IA puisse travailler avec des nombres. Il inclut aussi des "jokers" pour gérer les longueurs de phrases et les mots rares.

In [ ]:
VOCSIZE = 10000
w2idx = {}
idx2w = {}
w2idx["<pad>"]  = 0
w2idx["<unk>"] = 1
idx2w[1]="<unk>"
idx2w[0]="<pad>"

for i in range(VOCSIZE):
    w, _ = orderedvocab[i]
    w2idx[w] = i+2 # 0 et 1 sont déjà pris par <pad> et <unk>
    idx2w[i+2] = w

print(len(w2idx), " == ",len(idx2w))
for i in range(1,6):
    print(i, idx2w[i], w2idx[idx2w[i]])

10002  ==  10002
1 <unk> 1
2 ! 2
3 the 3
4 a 4
5 of 5


`<pad>` (pour padding) est un mot spécial. Nous en avons parlé quand nous avons discuté de "dimensionner les entrées". Il sera utilisé pour remplir (ou "padder") les phrases plus courtes afin qu'elles aient toutes la même longueur maximale. On lui attribue généralement l'index 0.

`<unk>` (pour unknown) est un autre mot spécial. Il représente tous les mots qui ne sont pas inclus dans notre VOCSIZE de 10 000 mots les plus fréquents, ou que nous n'avons jamais rencontrés pendant la phase d'apprentissage. Tous ces mots "inconnus" seront traités comme le même <unk>` par le modèle. On lui attribue l'index 1.

In [ ]:
NB_SENTENCES = 100000 # for each class
txtidx = []
maxlength = 0
for sent in postxt[
    1:NB_SENTENCES+1]+negtxt[ # la première est généralement utilisée pour des tests ou peut être vide/spéciale
        :NB_SENTENCES]:
    maxlength = max( # met à jour maxlength si la phrase actuelle est plus longue que toutes celles vues jusqu'à présent
        maxlength,len(
            sent))
    isent=[]
    for w in sent:
        widx=1 # si un mot n'est pas dans notre vocabulaire w2idx (les 10 000 mots les plus fréquents), il sera traité comme un mot inconnu.
        if w in w2idx:
            widx=w2idx[w]
        isent.append(widx)
    txtidx.append(
        th.LongTensor( # Une fois que tous les mots de la phrase sent ont été convertis en leurs index numériques dans isent, cette liste est transformée en un torch.LongTensor, type de tenseur PyTorch qui est optimisé pour stocker des entiers
            (isent)))

print(
    len(
        w2idx), " words in the vocab") # taille de notre vocabulaire final (avec <pad> et <unk>)
print(
    len(
        txtidx), " sentences") # 100 000 exemples pour chaque catégorie (positif et négatif)
print(maxlength, " is maximum sentence length")
print(
    txtidx[
        0]) # première phrase du dataset, mais sous sa forme numérisée (une séquence d'index)


### For the labels
labels = th.ones(
    [2*NB_SENTENCES]) # avons mis toutes les critiques positives d'abord, puis toutes les critiques négatives
labels[
    0:NB_SENTENCES] = 0 # première moitié du tableau labels contient des 0 (pour les critiques positives)

10002  words in the vocab
200000  sentences
48  is maximum sentence length
tensor([ 36,  25, 381,  10,  58,  21,  83])


In [ ]:
def idx2wordlist(idx_array):
    l = []
    for i in idx_array:
        l.append(idx2w[i.item()])
    return l
print(
    txtidx[
        0], txtidx[
            0].shape) # ce que votre IA va réellement voir

print(
    idx2wordlist(
        txtidx[
            0])) # phrase sous sa forme textuelle originale après avoir été numérisée et reconvertie
print(
    postxt[
        1]) # vraie phrase originale telle qu'elle a été chargée et nettoyée depuis le fichier imdb.pos

tensor([ 36,  25, 381,  10,  58,  21,  83]) torch.Size([7])
['do', "n't", 'miss', 'it', 'if', 'you', 'can']
['do', "n't", 'miss', 'it', 'if', 'you', 'can']


In [ ]:
pack = (txtidx, labels, idx2w)
# txtidx (vos phrases transformées en séquences d'index numériques),
# labels (les étiquettes positif/négatif correspondantes), et
# idx2w (le dictionnaire qui permet de retrouver les mots à partir de leurs index).

import pickle # transformer des objets Python complexes en un flux d'octets

if True :
     pickle.dump(pack, open('imdb-200k', 'wb'))

Pour l'instant, nous avons seulement la traduction mot -> index : Ce notebook réalise de la tokenisation, suivie de numérisation.
